In [ ]:
import torch
import requests
from PIL import Image
from io import BytesIO
from torchvision import transforms
from torchvision.models import resnet50

# Load the pre-trained ResNet50 model
model = resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode
for param in model.parameters():
    param.requires_grad = False  # Freeze the parameters

# Image URL
url = 'https://lionsvalley.co.za/wp-content/uploads/2015/11/africanelephant-square.jpg'

# Fetch the image
response = requests.get(url, stream=True)

if response.status_code == 200:
    # Check if the content is an image
    if "image" in response.headers["Content-Type"]:
        image_data = BytesIO(response.content)  # Convert the response to a file-like object
        try:
            original_image = Image.open(image_data).convert('RGB')  # Open image using PIL
            original_image.show()  # Optionally display the image
        except UnidentifiedImageError:
            print("Error: Unable to identify the image file.")
            exit()
    else:
        print(f"Error: The URL does not point to an image. Content-Type: {response.headers['Content-Type']}")
        exit()
else:
    print(f"Failed to fetch image. Status code: {response.status_code}")
    exit()

# Define image transformations (resize, tensor conversion, normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50 expects 224x224 images
    transforms.ToTensor(),  # Convert to tensor and scale to [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet stats
])

# Apply the transformations
image_tensor = transform(original_image).unsqueeze(0)  # Add batch dimension

# Forward pass through the model
with torch.no_grad():
    output = model(image_tensor)

print("Model output shape:", output.shape)  # Expected output: (1, 1000)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Failed to fetch image. Status code: 404


NameError: name 'original_image' is not defined

In [ ]:
image_net_classes= ''
image_net_ids=eval(image_net_classes)
image_net_classes={i:j for j, i in image_net_ids.item()}

In [ ]:
#@ function to normalize(img2tensor) and denormalize(tensor2img):
from torchvision import transforms as T
from torch.nn import functional as f

normalize=T.Normalize([0.485, 0.456, 0.406])

denormalize=T.Normalize(
    [-0.485/0.229, -0.456/0.224, -0.406/0.225], #reversing mean normalization
    [1/0.229, 1/0.224, 1/0.225] #reversing std
)

def Image2tensor(input):
  x=normalize(input.clone().permute(2, 0, 1)/255.)[None]
  return x

def tensor2Image(input):
  x=(denormalize(input[0].clone()).permute(1, 2, 0)*255.).type(torch.uint8)
  return x

In [ ]:
def predict_on_target(input):
  model.eval()
  show(input)
  input=Image2tensor(input)
  pred=model(input)
  pred=F.softmax(pred, dim=-1)[0]
  prob, clss=torch.max(pred, 0)
  clss=image_net_ids[clss.item()]
  print(f'PREDICTION: {clss} @ {prob.item()}')

In [ ]:
from tqdm import trange

losses=[]

def attack(image, model, target, epsilon=1e-6):
  input=Image2tensor(image)
  input.requires_grad=True
  pred=model(input)
  loss=nn.CrossEntropyLoss()(pred, target)
  loss.backward()
  losses.append(loss.mean().item())
  output= input - epsilon * input.grad.sign()
  output=tensor2Image(output)
  del input
  return output.detach()




In [ ]:
modified_images=[]
desired_targets=['lemon', 'comic book', 'sax, saxophone']

for target in desired_targets:
  target=torch.tensor([image_net_classes[target]])
  image_to_attack=original_image.clone()

  for _ in trange(10):
    image_to_attack=attack(image_to_attack, model, target)
  modified_images.append(image_to_attack)

for image in [original_image, *modified_images]:
  predict_on_image(image)
  inspect(image)